## Agents

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [61]:
import os
from dotenv import load_dotenv
load_dotenv()

AZURE_DEPLOYMENT_NAME= os.environ['AZURE_DEPLOYMENT_NAME']
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_KEY"] = os.environ['AZURE_OPEN_API_KEY']
os.environ["OPENAI_API_BASE"] = os.environ['AZURE_API_BASE']
os.environ["OPENAI_VERSION"] = "2023-03-15-preview"   


In [52]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [53]:
from langchain.document_loaders import TextLoader
loader = TextLoader('data/state_of_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)




Created a chunk of size 1067, which is longer than the specified 1000
Created a chunk of size 1007, which is longer than the specified 1000
Created a chunk of size 1307, which is longer than the specified 1000


In [70]:
embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002",chunk_size=1)
state_of_union_store = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")


Using embedded DuckDB without persistence: data will be transient


////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200


In [55]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")
docs = loader.load()

ruff_texts = text_splitter.split_documents(docs)
ruff_store = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")

Created a chunk of size 1356, which is longer than the specified 1000
Created a chunk of size 1923, which is longer than the specified 1000
Created a chunk of size 1267, which is longer than the specified 1000
Using embedded DuckDB without persistence: data will be transient


////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200
////////////
200


In [71]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    client=None,
    deployment_name=AZURE_DEPLOYMENT_NAME,
    engine=AZURE_DEPLOYMENT_NAME,
    openai_api_base=os.environ['AZURE_API_BASE'],
    openai_api_version="2023-03-15-preview",
    openai_api_key=os.environ['AZURE_OPEN_API_KEY'],
    temperature=0,
    request_timeout=180)



In [72]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)

vectorstore_info = VectorStoreInfo(
    name="state_of_union_address",
    description="the most recent state of the Union adress",
    vectorstore=state_of_union_store
)

toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)

agent_executor = create_vectorstore_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

USING API_BASE: 
https://tresen-test-1.openai.azure.com/


In [75]:
# agent_executor.run("What did biden say about ketanji brown jackson is the state of the union address?")